In [1]:
def ICON_scheduling(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False,scheduling=False):
    # nbMachines: number of machine
    # nbTasks: number of task
    # nb resources: number of resources
    # MC[m][r] resource capacity of machine m for resource r 
    # U[f][r] resource use of task f for resource r
    # D[f] duration of tasks f
    # E[f] earliest start of task f
    # L[f] latest end of task f
    # P[f] power use of tasks f
    # idle[m] idle cost of server m
    # up[m] startup cost of server m
    # down[m] shut-down cost of server m
    # q time resolution

    Machines = range(nbMachines)
    Tasks = range(nbTasks)
    Resources = range(nbResources)
    N = 1440//q

    M = Model("icon")
    if not verbose:
        M.setParam('OutputFlag', 0)

    x = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x"+str(f)+"_"+str(m)+"_"+str(t))
    x1 = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x1[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

    M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
    # earliest start time constraint
    M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
    # latest end time constraint
    M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f],N) )
    # constraint ensuring one task can have one start
    M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N)))
    M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
    # cosntraint for duration satisfying
    M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
    # machine variables
    v = {}
    for m in Machines:
        for t in range(N):
            v[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "v"+str(m)+"_"+str(t))
    y= {}  # start variable
    for m in Machines:
        for t in range(N):
            y[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "y"+str(m)+"_"+str(t))

    z = {} # shutdown variable
    for m in Machines:
        for t in range(N):
            z[(m,t)] = M.addVar(vtype=GRB.BINARY,name = "z"+str(m)+"_"+str(t))
    M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
    M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
    M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
    M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
    # capacity requirement
    M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
    # constraint for ensuring machine is on if a task runs on it 
    M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

    #M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
    M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  
                        for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
    
    M.optimize()
    schedule ={}
    if M.status == GRB.Status.OPTIMAL:
        m_on = M.getAttr('x',y)
        schedule["y"]= m_on

        m_off = M.getAttr('x',z)
        schedule["z"] = m_off

        task_on = M.getAttr('x',x)


        task = M.getAttr('x',x1)
        schedule["x1"]=task
        machine_run = M.getAttr('x',v)
        schedule["v"] = machine_run
        if verbose:
            for k,val in m_on.items():
                if int(val)>0:
                    print("Machine_m %d on at %d" %( k[0],k[1]))
            for k,val in m_off.items():
                if int(val)>0:
                    print("Machine_m %d off at %d" %( k[0],k[1]))
            for k,val in task_on.items():
                if int(val)>0:
                    print("Task_%d starts on machine_%d at %d"%(k[0],k[1],k[2]))
            print('\nCost: %g' % M.objVal)

        solver = np.zeros(N)
        '''
        for t in range(N+1):
        solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
        sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
        sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
        sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
        '''
        for t in range(N):
            solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 
        #return schedule, solver
        if scheduling:
            solver = np.zeros((nbTasks,nbMachines,N))
            for f in Tasks:
                for m in Machines:
                    for t in range(N):
                        solver[f,m,t] = task[(f,m,t)]
        return solver
    elif M.status == GRB.Status.INF_OR_UNBD:
        print('Model is infeasible or unbounded')

    elif M.status == GRB.Status.INFEASIBLE:
        print('Model is infeasible')
    elif M.status == GRB.Status.UNBOUNDED:
        print('Model is unbounded')

    else:
        print('Optimization ended with status %d' % M.status)

    return None


def ICON_scheduling_relaxation(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False,scheduling=False):
    # nbMachines: number of machine
    # nbTasks: number of task
    # nb resources: number of resources
    # MC[m][r] resource capacity of machine m for resource r 
    # U[f][r] resource use of task f for resource r
    # D[f] duration of tasks f
    # E[f] earliest start of task f
    # L[f] latest end of task f
    # P[f] power use of tasks f
    # idle[m] idle cost of server m
    # up[m] startup cost of server m
    # down[m] shut-down cost of server m
    # q time resolution

    Machines = range(nbMachines)
    Tasks = range(nbTasks)
    Resources = range(nbResources)
    N = 1440//q

    M = Model("icon")
    if not verbose:
        M.setParam('OutputFlag', 0)
    lb =0.0
    ub =1.0  
    x = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x[(f,m,t)] = M.addVar(lb,ub,name= "x"+str(f)+"_"+str(m)+"_"+str(t))

    x1 = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x1[(f,m,t)] = M.addVar(lb,ub,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

    M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
    # earliest start time constraint
    M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
    # latest end time constraint
    M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f],N) )
    # constraint ensuring one task can have one start
    M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N)))
    M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
    # cosntraint for duration satisfying
    M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
    # machine variables
    v = {}
    for m in Machines:
        for t in range(N):
            v[(m,t)] = M.addVar(lb,ub,name= "v"+str(m)+"_"+str(t))
    y= {}  # start variable
    for m in Machines:
        for t in range(N):
            y[(m,t)] = M.addVar(lb,ub,name= "y"+str(m)+"_"+str(t))

    z = {} # shutdown variable
    for m in Machines:
        for t in range(N):
            z[(m,t)] = M.addVar(lb,ub,name = "z"+str(m)+"_"+str(t))
    M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
    M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
    M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
    M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
    # capacity requirement
    M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
    # constraint for ensuring machine is on if a task runs on it 
    M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

    #M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
    M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  
                        for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
    #M = M.relax()
    M.optimize()
    schedule ={}
    if M.status == GRB.Status.OPTIMAL:

        m_on = M.getAttr('x',y)
        schedule["y"]= m_on

        m_off = M.getAttr('x',z)
        schedule["z"] = m_off

        task_on = M.getAttr('x',x)


        task = M.getAttr('x',x1)
        schedule["x1"]=task
        machine_run = M.getAttr('x',v)
        schedule["v"] = machine_run
        if verbose:
            for k,val in m_on.items():
                if val>0:
                    print("Machine_m %d on at %d" %( k[0],k[1]))
            for k,val in m_off.items():
                if val>0:
                    print("Machine_m %d off at %d" %( k[0],k[1]))
            for k,val in task_on.items():
                if val>0:
                    print("Task_%d starts on machine_%d at %d and runs for duration %d"%(k[0],k[1],k[2],D[k[0]]))
            print('\nCost: %g' % M.objVal)

        solver = np.zeros(N)
        '''
        for t in range(N+1):
        solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
        sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
        sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
        sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
        '''
        for t in range(N):
            solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60
        #return schedule, solver
        if scheduling:
            solver = np.zeros((nbTasks,nbMachines,N))
            for f in Tasks:
                for m in Machines:
                    for t in range(N):
                        solver[f,m,t] = task[(f,m,t)]            
        return  solver
    elif M.status == GRB.Status.INF_OR_UNBD:
        print('Model is infeasible or unbounded')

    elif M.status == GRB.Status.INFEASIBLE:
        print('Model is infeasible')
    elif M.status == GRB.Status.UNBOUNDED:
        print('Model is unbounded')

    else:
        print('Optimization ended with status %d' % M.status)

    return None

In [38]:
def ICON_scheduling2(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False):
	# nbMachines: number of machine
	# nbTasks: number of task
	# nb resources: number of resources
	# MC[m][r] resource capacity of machine m for resource r 
	# U[f][r] resource use of task f for resource r
	# D[f] duration of tasks f
	# E[f] earliest start of task f
	# L[f] latest end of task f
	# P[f] power use of tasks f
	# idle[m] idle cost of server m
	# up[m] startup cost of server m
	# down[m] shut-down cost of server m
	# q time resolution

	Machines = range(nbMachines)
	Tasks = range(nbTasks)
	Resources = range(nbResources)
	N = 1440//q

	M = Model("icon")
	M.setParam('OutputFlag', 0)

	x = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N):
				x[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x"+str(f)+"_"+str(m)+"_"+str(t))

	x1 = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N):
				x1[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

	M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
	# earliest start time constraint
	M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
	# latest end time constraint
	M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f]+1,N) )
	# constraint ensuring one task can have one start
	M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) 
                  for f in Tasks for  m in Machines for t in range(1,N)))
	M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
	# cosntraint for duration satisfying
	M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
	# machine variables
	v = {}
	for m in Machines:
		for t in range(N):
			v[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "v"+str(m)+"_"+str(t))
	y= {}  # start variable
	for m in Machines:
		for t in range(N):
			y[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "y"+str(m)+"_"+str(t))

	z = {} # shutdown variable
	for m in Machines:
		for t in range(N):
			z[(m,t)] = M.addVar(vtype=GRB.BINARY,name = "z"+str(m)+"_"+str(t))
	M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
	M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
	M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
	M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
	# capacity requirement
	M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
	# constraint for ensuring machine is on if a task runs on it 
	M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

	#M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
	M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 ) for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
	M.optimize()
	schedule ={}
	if M.status == GRB.Status.OPTIMAL:
		
		m_on = M.getAttr('x',y)
		schedule["y"]= m_on
	
		m_off = M.getAttr('x',z)
		schedule["z"] = m_off
	
		task_on = M.getAttr('x',x)

	
		task = M.getAttr('x',x1)
		schedule["x1"]=task
		machine_run = M.getAttr('x',v)
		schedule["v"] = machine_run
		if verbose:
		
			for k,val in m_on.items():
				if int(val)>0:
					print("Machine_m %d on at %d" %( k[0],k[1]))
			for k,val in m_off.items():
				if int(val)>0:
					print("Machine_m %d off at %d" %( k[0],k[1]))
			for k,val in task_on.items():
				if int(val)>0:
					print("Task_%d starts on machine_%d at %d and runs for duration %d"%(k[0],k[1],k[2],D[k[0]]))
			print('\nCost: %g' % M.objVal)
		
		solver = np.zeros(N)
		'''
		for t in range(N+1):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
			sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
			sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
			sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
		'''
		for t in range(N):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 
		#return schedule, solver
		return solver
	elif M.status == GRB.Status.INF_OR_UNBD:
		print('Model is infeasible or unbounded')
    
	elif M.status == GRB.Status.INFEASIBLE:
		print('Model is infeasible')
	elif M.status == GRB.Status.UNBOUNDED:
		print('Model is unbounded')

	else:
		print('Optimization ended with status %d' % M.status)

	return None

In [18]:
def ICON_scheduling2_relaxation(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False):
	# nbMachines: number of machine
	# nbTasks: number of task
	# nb resources: number of resources
	# MC[m][r] resource capacity of machine m for resource r 
	# U[f][r] resource use of task f for resource r
	# D[f] duration of tasks f
	# E[f] earliest start of task f
	# L[f] latest end of task f
	# P[f] power use of tasks f
	# idle[m] idle cost of server m
	# up[m] startup cost of server m
	# down[m] shut-down cost of server m
	# q time resolution

	Machines = range(nbMachines)
	Tasks = range(nbTasks)
	Resources = range(nbResources)
	N = 1440//q

	M = Model("icon")
	M.setParam('OutputFlag', 0)
	lb =0.0
	ub =1.0  
	x = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N+1):
				x[(f,m,t)] = M.addVar(lb,ub,name= "x"+str(f)+"_"+str(m)+"_"+str(t))
  
	x1 = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N+1):
				x1[(f,m,t)] = M.addVar(lb,ub,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

	M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
	# earliest start time constraint
	M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
	# latest end time constraint
	M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f]+1,N+1) )
	# constraint ensuring one task can have one start
	M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N+1)))
	M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N+1) for m in Machines) == 1  for f in Tasks))
	# cosntraint for duration satisfying
	M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N+1) for m in Machines) == D[f]  for f in Tasks))
	# machine variables
	v = {}
	for m in Machines:
		for t in range(N+1):
			v[(m,t)] = M.addVar(lb,ub,name= "v"+str(m)+"_"+str(t))
	y= {}  # start variable
	for m in Machines:
		for t in range(N+1):
			y[(m,t)] = M.addVar(lb,ub,name= "y"+str(m)+"_"+str(t))

	z = {} # shutdown variable
	for m in Machines:
		for t in range(N+1):
			z[(m,t)] = M.addVar(lb,ub,name = "z"+str(m)+"_"+str(t))
	M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
	M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N+1))
	M.addConstrs((z[(m,N)]== v[(m,N)]  ) for m in Machines )
	M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N))
	# capacity requirement
	M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N+1) )
	# constraint for ensuring machine is on if a task runs on it 
	M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N+1))

	#M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
	M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
	#M = M.relax()
	M.optimize()
	schedule ={}
	if M.status == GRB.Status.OPTIMAL:
		
		m_on = M.getAttr('x',y)
		schedule["y"]= m_on
	
		m_off = M.getAttr('x',z)
		schedule["z"] = m_off
	
		task_on = M.getAttr('x',x)

	
		task = M.getAttr('x',x1)
		schedule["x1"]=task
		machine_run = M.getAttr('x',v)
		schedule["v"] = machine_run
		if verbose:
		
			for k,val in m_on.items():
				if val>0:
					print("Machine_m %d on at %d" %( k[0],k[1]))
			for k,val in m_off.items():
				if val>0:
					print("Machine_m %d off at %d" %( k[0],k[1]))
			for k,val in task_on.items():
				if val>0:
					print("Task_%d starts on machine_%d at %d and runs for duration %d"%(k[0],k[1],k[2],D[k[0]]))
			print('\nCost: %g' % M.objVal)
		
		solver = np.zeros(N+1)
		'''
		for t in range(N+1):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
			sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
			sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
			sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
		'''
		for t in range(N+1):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 
		return schedule, solver
		#return solver
	elif M.status == GRB.Status.INF_OR_UNBD:
		print('Model is infeasible or unbounded')
    
	elif M.status == GRB.Status.INFEASIBLE:
		print('Model is infeasible')
	elif M.status == GRB.Status.UNBOUNDED:
		print('Model is unbounded')

	else:
		print('Optimization ended with status %d' % M.status)

	return None

In [5]:
def ICON_scheduling(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False):
	# nbMachines: number of machine
	# nbTasks: number of task
	# nb resources: number of resources
	# MC[m][r] resource capacity of machine m for resource r 
	# U[f][r] resource use of task f for resource r
	# D[f] duration of tasks f
	# E[f] earliest start of task f
	# L[f] latest end of task f
	# P[f] power use of tasks f
	# idle[m] idle cost of server m
	# up[m] startup cost of server m
	# down[m] shut-down cost of server m
	# q time resolution

	Machines = range(nbMachines)
	Tasks = range(nbTasks)
	Resources = range(nbResources)
	N = 1440//q

	M = Model("icon")
	if not verbose:    
		M.setParam('OutputFlag', 0)

	x = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N):
				x[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x"+str(f)+"_"+str(m)+"_"+str(t))

	x1 = {}
	for f in Tasks:
		for m in Machines:
			for t in range(N):
				x1[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

	M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
	# earliest start time constraint
	M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
	# latest end time constraint
	M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f],N) )
	# constraint ensuring one task can have one start
	M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N)))
	M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
	# cosntraint for duration satisfying
	M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
	# machine variables
	v = {}
	for m in Machines:
		for t in range(N):
			v[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "v"+str(m)+"_"+str(t))
	y= {}  # start variable
	for m in Machines:
		for t in range(N):
			y[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "y"+str(m)+"_"+str(t))

	z = {} # shutdown variable
	for m in Machines:
		for t in range(N):
			z[(m,t)] = M.addVar(vtype=GRB.BINARY,name = "z"+str(m)+"_"+str(t))
	M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
	M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
	M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
	M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
	# capacity requirement
	M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
	# constraint for ensuring machine is on if a task runs on it 
	M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

	#M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
	M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  
		for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
	M.optimize()
	schedule ={}
	if M.status == GRB.Status.OPTIMAL:
		
		m_on = M.getAttr('x',y)
		schedule["y"]= m_on
	
		m_off = M.getAttr('x',z)
		schedule["z"] = m_off
	
		task_on = M.getAttr('x',x)

	
		task = M.getAttr('x',x1)
		schedule["x1"]=task
		machine_run = M.getAttr('x',v)
		schedule["v"] = machine_run
		if verbose:
		
			for k,val in m_on.items():
				if int(val)>0:
					print("Machine_m %d on at %d" %( k[0],k[1]))
			for k,val in m_off.items():
				if int(val)>0:
					print("Machine_m %d off at %d" %( k[0],k[1]))
			for k,val in task_on.items():
				if int(val)>0:
					print("Task_%d starts on machine_%d at %d"%(k[0],k[1],k[2]))
			print('\nCost: %g' % M.objVal)
		
		solver = np.zeros(N)
		'''
		for t in range(N+1):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
			sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
			sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
			sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
		'''
		for t in range(N):
			solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 
		#return schedule, solver
		return solver
	elif M.status == GRB.Status.INF_OR_UNBD:
		print('Model is infeasible or unbounded')
    
	elif M.status == GRB.Status.INFEASIBLE:
		print('Model is infeasible')
	elif M.status == GRB.Status.UNBOUNDED:
		print('Model is unbounded')

	else:
		print('Optimization ended with status %d' % M.status)

	return None

In [21]:
def ICON_scheduling(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False,scheduling=False):
    # nbMachines: number of machine
    # nbTasks: number of task
    # nb resources: number of resources
    # MC[m][r] resource capacity of machine m for resource r 
    # U[f][r] resource use of task f for resource r
    # D[f] duration of tasks f
    # E[f] earliest start of task f
    # L[f] latest end of task f
    # P[f] power use of tasks f
    # idle[m] idle cost of server m
    # up[m] startup cost of server m
    # down[m] shut-down cost of server m
    # q time resolution

    Machines = range(nbMachines)
    Tasks = range(nbTasks)
    Resources = range(nbResources)
    N = 1440//q

    M = Model("icon")
    if not verbose:
        M.setParam('OutputFlag', 0)

    x = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x"+str(f)+"_"+str(m)+"_"+str(t))
    x1 = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x1[(f,m,t)] = M.addVar(vtype=GRB.BINARY,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

    M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
    # earliest start time constraint
    M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
    # latest end time constraint
    M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f],N) )
    # constraint ensuring one task can have one start
    M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N)))
    M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
    # cosntraint for duration satisfying
    M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
    # machine variables
    v = {}
    for m in Machines:
        for t in range(N):
            v[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "v"+str(m)+"_"+str(t))
    y= {}  # start variable
    for m in Machines:
        for t in range(N):
            y[(m,t)] = M.addVar(vtype=GRB.BINARY,name= "y"+str(m)+"_"+str(t))

    z = {} # shutdown variable
    for m in Machines:
        for t in range(N):
            z[(m,t)] = M.addVar(vtype=GRB.BINARY,name = "z"+str(m)+"_"+str(t))
    M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
    M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
    M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
    M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
    # capacity requirement
    M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
    # constraint for ensuring machine is on if a task runs on it 
    M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

    #M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
    M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  
                        for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
    M.optimize()
    schedule ={}
    if M.status == GRB.Status.OPTIMAL:
        m_on = M.getAttr('x',y)
        schedule["y"]= m_on

        m_off = M.getAttr('x',z)
        schedule["z"] = m_off

        task_on = M.getAttr('x',x)


        task = M.getAttr('x',x1)
        schedule["x1"]=task
        machine_run = M.getAttr('x',v)
        schedule["v"] = machine_run
        if verbose:
            for k,val in m_on.items():
                if int(val)>0:
                    print("Machine_m %d on at %d" %( k[0],k[1]))
            for k,val in m_off.items():
                if int(val)>0:
                    print("Machine_m %d off at %d" %( k[0],k[1]))
            for k,val in task_on.items():
                if int(val)>0:
                    print("Task_%d starts on machine_%d at %d"%(k[0],k[1],k[2]))
            print('\nCost: %g' % M.objVal)

        solver = np.zeros(N)
        '''
        for t in range(N+1):
        solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
        sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
        sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
        sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
        '''
        for t in range(N):
            solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 
        #return schedule, solver
        if scheduling:
            solver = np.zeros((nbTasks,nbMachines,N))
            for f in Tasks:
                for m in Machines:
                    for t in range(N):
                        solver[f,m,t] = task[(f,m,t)]
        return solver
    elif M.status == GRB.Status.INF_OR_UNBD:
        print('Model is infeasible or unbounded')

    elif M.status == GRB.Status.INFEASIBLE:
        print('Model is infeasible')
    elif M.status == GRB.Status.UNBOUNDED:
        print('Model is unbounded')

    else:
        print('Optimization ended with status %d' % M.status)

    return None


def ICON_scheduling_relaxation(price,nbMachines,nbTasks,nbResources,MC,U,D,E,L,P,idle,up,down,q,verbose=False,scheduling=False):
    # nbMachines: number of machine
    # nbTasks: number of task
    # nb resources: number of resources
    # MC[m][r] resource capacity of machine m for resource r 
    # U[f][r] resource use of task f for resource r
    # D[f] duration of tasks f
    # E[f] earliest start of task f
    # L[f] latest end of task f
    # P[f] power use of tasks f
    # idle[m] idle cost of server m
    # up[m] startup cost of server m
    # down[m] shut-down cost of server m
    # q time resolution

    Machines = range(nbMachines)
    Tasks = range(nbTasks)
    Resources = range(nbResources)
    N = 1440//q

    M = Model("icon")
    if not verbose:
        M.setParam('OutputFlag', 0)
    lb =0.0
    ub =1.0  
    x = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x[(f,m,t)] = M.addVar(lb,ub,name= "x"+str(f)+"_"+str(m)+"_"+str(t))

    x1 = {}
    for f in Tasks:
        for m in Machines:
            for t in range(N):
                x1[(f,m,t)] = M.addVar(lb,ub,name= "x1"+str(f)+"_"+str(m)+"_"+str(t))

    M.addConstrs((x[(f,m,0)]== x1[(f,m,0)]  ) for f in Tasks for  m in Machines )
    # earliest start time constraint
    M.addConstrs( (x[(f,m,t)]==0 )  for f in Tasks for m in Machines for t in range(E[f]) )
    # latest end time constraint
    M.addConstrs( (x1[(f,m,t)]==0 )   for f in Tasks for m in Machines for t in range(L[f],N) )
    # constraint ensuring one task can have one start
    M.addConstrs((x[(f,m,t)]>= (x1[(f,m,t)] - x1[(f,m,t-1)] ) for f in Tasks for  m in Machines for t in range(1,N)))
    M.addConstrs(( quicksum(x[(f,m,t)] for t in range(N) for m in Machines) == 1  for f in Tasks))
    # cosntraint for duration satisfying
    M.addConstrs(( quicksum(x1[(f,m,t)] for t in range(N) for m in Machines) == D[f]  for f in Tasks))
    # machine variables
    v = {}
    for m in Machines:
        for t in range(N):
            v[(m,t)] = M.addVar(lb,ub,name= "v"+str(m)+"_"+str(t))
    y= {}  # start variable
    for m in Machines:
        for t in range(N):
            y[(m,t)] = M.addVar(lb,ub,name= "y"+str(m)+"_"+str(t))

    z = {} # shutdown variable
    for m in Machines:
        for t in range(N):
            z[(m,t)] = M.addVar(lb,ub,name = "z"+str(m)+"_"+str(t))
    M.addConstrs((y[(m,0)]== v[(m,0)]  ) for m in Machines )
    M.addConstrs((y[(m,t)]>= (v[(m,t)] - v[(m,t-1)] )) for  m in Machines for t in range(1,N))
    M.addConstrs((z[(m,N-1)]== v[(m,N-1)]  ) for m in Machines )
    M.addConstrs((z[(m,t)]>= (v[(m,t)] - v[(m,t+1)] )) for  m in Machines for t in range(N-1))
    # capacity requirement
    M.addConstrs( (sum(x1[(f,m,t)]*U[f][r] for f in Tasks ) <= MC[m][r]) for m in Machines for r in Resources for t in range(N) )
    # constraint for ensuring machine is on if a task runs on it 
    M.addConstrs ( x1[(f,m,t)] <= v[(m,t)] for f in Tasks for  m in Machines for t in range(N))

    #M.setObjective( sum(( y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 +  (v[(m,t)]*idle[m]*price[t])*q/60 + (y[(m,t)]*up[m]) + (z[(m,t)]*down[m]))  for f in Tasks for m in Machines for t in range(N+1)) , GRB.MINIMIZE)
    M.setObjective( sum(( v[(m,t)] + y[(m,t)] +z[(m,t)]+ (x1[(f,m,t)]*price[t]*P[f])*q/60 )  
                        for f in Tasks for m in Machines for t in range(N)) , GRB.MINIMIZE)
    #M = M.relax()
    M.optimize()
    schedule ={}
    if M.status == GRB.Status.OPTIMAL:

        m_on = M.getAttr('x',y)
        schedule["y"]= m_on

        m_off = M.getAttr('x',z)
        schedule["z"] = m_off

        task_on = M.getAttr('x',x)


        task = M.getAttr('x',x1)
        schedule["x1"]=task
        machine_run = M.getAttr('x',v)
        schedule["v"] = machine_run
        if verbose:
            for k,val in m_on.items():
                if val>0:
                    print("Machine_m %d on at %d" %( k[0],k[1]))
            for k,val in m_off.items():
                if val>0:
                    print("Machine_m %d off at %d" %( k[0],k[1]))
            for k,val in task_on.items():
                if val>0:
                    print("Task_%d starts on machine_%d at %d and runs for duration %d"%(k[0],k[1],k[2],D[k[0]]))
            print('\nCost: %g' % M.objVal)

        solver = np.zeros(N)
        '''
        for t in range(N+1):
        solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60 + \
        sum( machine_run[(m,t)]*idle[m] for m in  Machines)*q/60 + \
        sum(m_on[(m,t)]*up[m] for m in Machines)/price[t] + \
        sum(m_off[(m,t)]*down[m] for m in Machines)/price[t]
        '''
        for t in range(N):
            solver[t] = sum(task[(f,m,t)]*P[f] for f in Tasks for m in Machines)*q/60
        #return schedule, solver
        if scheduling:
            solver = np.zeros((nbTasks,nbMachines,N))
            for f in Tasks:
                for m in Machines:
                    for t in range(N):
                        solver[f,m,t] = task[(f,m,t)]            
        return  solver
    elif M.status == GRB.Status.INF_OR_UNBD:
        print('Model is infeasible or unbounded')

    elif M.status == GRB.Status.INFEASIBLE:
        print('Model is infeasible')
    elif M.status == GRB.Status.UNBOUNDED:
        print('Model is unbounded')

    else:
        print('Optimization ended with status %d' % M.status)

    return None


In [199]:
from gurobipy import *
import time
import numpy as np
data = np.load('../Data.npz')

X_1gtrain = data['X_1gtrain']
X_1gtest = data['X_1gtest']
y_train = data['y_train']
y_test = data['y_test']

X_1gvalidation = X_1gtest[0:2880,:]
y_validation = y_test[0:2880]

y_test= y_test[2880:]
X_1gtest = X_1gtest[2880:,:]

weights = [data['weights'].tolist()]

weights = np.array(weights)
import time
import logging
formatter = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(filename='test1.log', level=logging.INFO,format=formatter)

logging.info("with start")

def solveKnapsackProblemRelaxation(profits, weights, capacity, time_limit=None,warmstart=None, use_dp=True):
    #assert len(profits) == len(weights)
    #print(profits)
    #assert(len([x for x in profits if x < 0]) == 0)
    #if len([x for x in profits if x < 0]) != 0:
        #print("WARNING: NEGATIVE PROFITS IN KNAPSACK SOLVING!\n\n!!!!!!!!!!\n\n")
        #print(profits)
        #print(weights)
    multi = (len(weights)>1)
    
    profits = [v for v in profits]
    weights = [[w for w in W] for W in weights]
    capacity = [c for c in capacity]
    start_time = time.time()
    n = len(profits)
    
    
    m = Model()
    #m.setParam('OutputFlag', 0)
    lb= [0.0]*n
    ub = [1.0]*n
    #x = m.addVars(n,ub=ub, name='x')
    x = {}
    for i in range(n):
        #x[(i)] = m.addVar(lb=0,ub=1, name= "x"+str(i))
        x[i] = m.addVar(vtype=GRB.BINARY,name= "x"+str(i))
    
    if warmstart is not None:
        for i in range(n):
            x[i].start = warmstart[i]
            m.update()
            print(x[i])
    m.update()
    '''
    for w in weights:
        for c in capacity:
            m.addConstr(x.prod(w) <= c)
    m.setObjective(x.prod(profits), GRB.MAXIMIZE) 
    '''
    m.setObjective(sum( (x[i]*profits[i]) for i in range(n)), GRB.MAXIMIZE)
    for w in weights:
        for c in capacity:    
            m.addConstr(( quicksum(x[i]*w[i] for i in range(n) ) <= c))
    m.optimize()
    

    solution_info = {}
    if (m.status == GRB.Status.OPTIMAL):
        solution_info['runtime'] = m.Runtime
        solution_info['objective'] = m.objVal
        m_on = m.getAttr('x',x)
        sol = list(m_on.values())     
        solution_info['assignments'] =  [i for i in sol]
    else:
        print("SOME EXCEPTION HAPPENED! RETURNING GARBAGE\n")
        val = 0
        import sys
            
        solution_info = {}
        solution_info['runtime'] = m.Runtime
        solution_info['objective'] = val
        solution_info['assignments'] = [0 for x in range(len(profits))]
    return solution_info
s = solveKnapsackProblemRelaxation(profits= y_test[0:48], weights=weights, capacity=[200])
sol = s['assignments']
s = solveKnapsackProblemRelaxation(profits= y_test[0:48], warmstart=sol, weights=weights, capacity=[200])

In [200]:
import numpy as np
data = np.load('../Data.npz')

X_1gtrain = data['X_1gtrain']
X_1gtest = data['X_1gtest']
y_train = data['y_train']
y_test = data['y_test']

X_1gvalidation = X_1gtest[0:2880,:]
y_validation = y_test[0:2880]

y_test= y_test[2880:]
X_1gtest = X_1gtest[2880:,:]

weights = [data['weights'].tolist()]

weights = np.array(weights)

In [201]:
weights

array([[5, 3, 3, 5, 5, 7, 7, 3, 7, 7, 3, 3, 5, 3, 7, 3, 7, 7, 5, 5, 3, 5,
        5, 3, 7, 7, 3, 7, 5, 5, 7, 3, 7, 3, 3, 5, 7, 5, 3, 5, 3, 7, 5, 7,
        5, 5, 3, 7]])

In [202]:
s = solveKnapsackProblemRelaxation(profits= y_test[0:48], weights=weights, capacity=[200])

Optimize a model with 1 rows, 48 columns and 48 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [3e+00, 7e+00]
  Objective range  [9e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective 12340.667637
Presolve time: 0.00s
Presolved: 1 rows, 48 columns, 48 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 1.319118e+04, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13191.1799    0    1 12340.6676 13191.1799  6.89%     -    0s
H    0     0                    13116.290638 13191.1799  0.57%     -    0s
H    0     0                    13186.591313 13191.1799  0.03%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01 seconds
Thread count was 8 (of 8 available processors)

Solution coun

In [203]:
sol = s['assignments']
sol

[1.0,
 1.0,
 1.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -0.0,
 1.0]

In [204]:
s = solveKnapsackProblemRelaxation(profits= y_test[0:48], warmstart=sol, weights=weights, capacity=[200])

<gurobi.Var x0>
<gurobi.Var x1>
<gurobi.Var x2>
<gurobi.Var x3>
<gurobi.Var x4>
<gurobi.Var x5>
<gurobi.Var x6>
<gurobi.Var x7>
<gurobi.Var x8>
<gurobi.Var x9>
<gurobi.Var x10>
<gurobi.Var x11>
<gurobi.Var x12>
<gurobi.Var x13>
<gurobi.Var x14>
<gurobi.Var x15>
<gurobi.Var x16>
<gurobi.Var x17>
<gurobi.Var x18>
<gurobi.Var x19>
<gurobi.Var x20>
<gurobi.Var x21>
<gurobi.Var x22>
<gurobi.Var x23>
<gurobi.Var x24>
<gurobi.Var x25>
<gurobi.Var x26>
<gurobi.Var x27>
<gurobi.Var x28>
<gurobi.Var x29>
<gurobi.Var x30>
<gurobi.Var x31>
<gurobi.Var x32>
<gurobi.Var x33>
<gurobi.Var x34>
<gurobi.Var x35>
<gurobi.Var x36>
<gurobi.Var x37>
<gurobi.Var x38>
<gurobi.Var x39>
<gurobi.Var x40>
<gurobi.Var x41>
<gurobi.Var x42>
<gurobi.Var x43>
<gurobi.Var x44>
<gurobi.Var x45>
<gurobi.Var x46>
<gurobi.Var x47>
Optimize a model with 1 rows, 48 columns and 48 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [3e+00, 7e+00]
  Objective range  [9e+01,

In [67]:
s

{'runtime': 0.0003490447998046875,
 'objective': 13191.179894211415,
 'assignments': [1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.5714285714285714,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0]}